In [310]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('Customers').getOrCreate()
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
import numpy as np
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [311]:
def bin2dec(x,th):
    dec=0;
    for i in range(0,4):
        dec=dec+x[3-i]*pow(2,i)
    if (dec>th):
        dec=th
    if (dec<2):
        dec=2
    return dec 

def selectedfeat(ga,th):
    a=[]
    for i in range(0,len(ga)-4):
        if (ga[i]==1):
            fn="_c" + str(i)
            a.append(fn)
    dec=bin2dec(ga[-4:],th)    
#     print(a)
#     print(dec)
    return a,dec

In [312]:
import random
def crossover(x,pc):
    noc=pc*np.shape(x)[0]
    if(noc%2!=0):
        noc=noc+1 
    a=random.sample(range(1, np.shape(x)[0]), int(noc))
    bp=int(np.shape(x)[1]/2)
    x1=np.zeros((int(noc/2),np.shape(x)[1]))
    for i in range(0,int(noc/2)):
        a1=a[2*i]
        a2=a[2*i+1]
        c1=x[a1,:]
        c2=x[a2,:]
        x1[i,0:bp]=c1[0:bp]
        x1[i,bp+1:]=c2[bp+1:]
    return x1 

In [313]:
def mutation(x,pm):
    noc=int(pm*np.shape(x)[1])
    a=np.random.randint(np.shape(x)[1], size=(np.shape(x)[0], noc))
    for i in range(0,np.shape(x)[0]):
        a1=a[i,:]
        for j in range(0,noc):
            if (x[i,a1[j]]==0):
                x[i,a1[j]]=1
            else:
                x[i,a1[j]]=0
    return x 

In [314]:
def makedata(ga,data,th):
    dataf=[]
    for row in ga:
        d=[]
        a,dec=selectedfeat(row,th)
        featureassembler=VectorAssembler(inputCols=a,outputCol="features")
        dataset=featureassembler.transform(data)
        d.append(dataset)
        d.append(dec)
        dataf.append(d)
    return dataf

In [315]:
def fitnessvaluega1(dataset1):
    dataset=dataset1[0]
    k=dataset1[1]
    dataset=dataset.select('features')
    dataset.rdd.collect()
    kmeans = KMeans().setK(int(k)).setSeed(1)
    model = kmeans.fit(dataset)
    predictions = model.transform(dataset)
    evaluator = ClusteringEvaluator()
    silhouette = evaluator.evaluate(predictions)
    return silhouette

In [316]:
import multiprocess 
import time 

In [317]:
# noc=input("Enter number of chromosome")
# noc=int(noc)
# th=input("Enter max threshold")
# th=int(th)
# wn=input("Enter number of working node")
# wn=int(wn)
# nog=input("Enter number of Generations")
# nog=int(nog)
noc =25
th = 0.5
wn = 4
nog = 20

In [318]:
from multiprocess.pool import ThreadPool 
pool=ThreadPool(wn)


In [319]:
import time
befit=-1
pc=0.5
pm=0.1

bsv=[]
timef=[]
befs=[]
for j in range(0,14):
    fname= str(j+1) +'.csv'
    dataset=spark.read.csv(fname,inferSchema=True,header=False)
    columns = len(dataset.columns)
    rows=dataset.count()
    print(rows,columns)
    ga=np.random.randint(2, size=(noc, columns+4))
    dataf=makedata(ga,dataset,th)
    iterbest=np.zeros((nog))
    tf=np.zeros((nog))
    
    for gno in range(0,nog):
        print(gno)
        start = time.time()
        s=np.zeros((noc))
        s=pool.map(fitnessvaluega1,dataf)
        maxs=np.max(s)
        maxi=np.argmax(s)  
        if maxs>befit:
            befit=maxs
            gaf=ga[maxi,:]
        iterbest[gno]=befit    
        ga=crossover(ga,pc)
        ga=mutation(ga,pm)
        nr=np.shape(ga)[0]   
        ga=np.concatenate((ga,np.random.randint(2, size=(noc-nr, columns+4))), axis=0)
        print(befit,gno)
        dataf=makedata(ga,dataset,th)
        end = time.time()
        tf[gno]=end-start
    bsv.append(iterbest)
    timef.append(tf)
    befs.append(befit)

26509 100
0
0.2880711068407523 0
1
0.3154051638763784 1
2
0.3154051638763784 2
3
0.3154051638763784 3
4
0.3154051638763784 4
5
0.3154051638763784 5
6
0.3154051638763784 6
7
0.3154051638763784 7
8
0.3154051638763784 8
9
0.3230892729472188 9
10
0.3230892729472188 10
11
0.3230892729472188 11
12
0.3230892729472188 12
13
0.3230892729472188 13
14
0.3230892729472188 14
15
0.3230892729472188 15
16
0.3230892729472188 16
17
0.3230892729472188 17
18
0.3230892729472188 18
19
0.3230892729472188 19
26509 100
0
0.5396931123655117 0
1
0.5444115841132842 1
2
0.5535325989000222 2
3
0.5535325989000222 3
4
0.5535325989000222 4
5


KeyboardInterrupt: 

In [299]:
bsv

[]

In [ ]:
start = time.time()
s=pool.map(fitnessvaluega1,dataf)
end = time.time()
print(end-start)

In [ ]:
pool=ThreadPool(1)
start = time.time()
s=pool.map(fitnessvaluega1,dataf)
end = time.time()
print(end-start)

In [ ]:
pool=ThreadPool(2)
start = time.time()
s=pool.map(fitnessvaluega1,dataf)
end = time.time()
print(end-start)

In [ ]:
pool=ThreadPool(3)
start = time.time()
s=pool.map(fitnessvaluega1,dataf)
end = time.time()
print(end-start)

In [ ]:
gnof=np.zeros(nog)
for i in range(0,nog):
    gnof[i]=i+1
import matplotlib.pyplot as plt 
plt.plot(gnof,bsv[0])
plt.xlabel('Generation Number') 
plt.ylabel('Fitness Value')  
plt.title('1') 
plt.show() 
plt.savefig

# plt.plot(gnof,bsv[1])
# plt.xlabel('Generation Number') 
# plt.ylabel('Fitness Value')  
# plt.title('Skg') 
# plt.show() 


In [ ]:
# plt.plot(gnof,timef[0])
# plt.xlabel('Generation Number') 
# plt.ylabel('Time Value')  
# plt.title('CBOW') 
# plt.show() 
# plt.savefig('timecbow.png')

# plt.plot(gnof,timef[1])
# plt.xlabel('Generation Number') 
# plt.ylabel('Time Value')  
# plt.title('Skg') 
# plt.show() 
# plt.savefig('timeskg.png')

In [ ]:
plt.plot(gnof,timef[0])
plt.xlabel('Generation Number') 
plt.ylabel('Time Value')  
plt.title('CBOW') 
plt.show() 




In [ ]:
plt.plot(gnof,timef[1])
plt.xlabel('Generation Number') 
plt.ylabel('Time Value')  
plt.title('Skg') 
plt.show() 
plt.savefig('timeskg_20.png')

In [ ]:
# plt.savefig('cbow_20.png')
# plt.savefig('skg_20.png')
# plt.savefig('timecbow_20.png')

In [ ]:
# plt.plot(gnof,timef[1])
# plt.xlabel('Generation Number') 
# plt.ylabel('Time Value')  
# plt.title('Skg') 
# plt.show() 
# plt.savefig('timeskg.png')

In [ ]:
# plt.plot(gnof,timef[1])
# plt.xlabel('Generation Number') 
# plt.ylabel('Time Value')  
# plt.title('Skg') 
# plt.show() 
# plt.savefig('timeskg.png')

In [ ]:
# plt.plot(gnof,timef[1])
# plt.xlabel('Generation Number') 
# plt.ylabel('Time Value')  
# plt.title('Skg') 
# plt.show() 
# plt.savefig('D:/data/CORD-19-research-challenge/data/timeskg.png')